In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.express as px
import plotly 
import warnings
warnings.filterwarnings("ignore")
import re

## Spatial Distribution

#### Data processing

In [2]:
df=pd.read_csv('data/all_keywords.csv')
df=df[['Date', 'Content', "User's location", 'userid']]
df=df.dropna()
df

,Date,Content,User's location,userid
8,2022-08-31 20:07:09+00:00,#BBNaija me just waiting for Daniella to break...,"Pimville Soweto, South Africa",4.472969e+08
11,2022-08-31 18:29:22+00:00,"Exhausted 🥱 no sleep since yesterday, I needs ...","Philadelphia, PA",2.432277e+09
12,2022-08-31 15:02:00+00:00,ICYMI: Lil' Miss #TeamNoSleep finally gets a f...,"Myrtle Beach, South Carolina",8.699900e+08
14,2022-08-31 12:22:23+00:00,I think I’m super-delirious. I do believe I ne...,"California, USA",1.801410e+07
16,2022-08-31 11:17:41+00:00,Budget Kraber is a must have in my load out 😘\...,Olympus,1.218736e+18
...,...,...,...,...
892019,2022-06-01 02:40:54+00:00,2010s revisionist westerns AKA the world's mos...,"Chicago, IL",6.332248e+08
892020,2022-06-01 02:35:57+00:00,Ight I took my sleep aid medicine I’ll be KO I...,"Atlanta, GA",4.524712e+08
892021,2022-06-01 02:25:48+00:00,@SpiroAgnewGhost What’s her next segment “Why ...,Hopping Time Zones in my G6,1.427527e+18
892022,2022-06-01 01:41:36+00:00,Having trouble sleeping? \n\nRogue Shop sleep ...,"Chapel Hill, NC",2.686267e+08


map the user's location to concise location <br>
We separate to 3 parts

In [4]:
geolocator = Nominatim(user_agent = "my_request")
for i in range(len(df)):
    try:
        location = geolocator.geocode(df["User's location"][i])
        map.append(location.address)
    except: 
        continue
map = pd.DataFrame(map)

This mapping needs extremely long time to run. <br>
The second part crashed accidentally. <br>
We got the result of the first part and tried to visualize it in the world scale <br>
But we found that the US part has too many tweets <br>
So we decided to visualize the map only in US area. <br>
Therefore, we apply the brutal search for the second part to find the US locations.

In [3]:
#US, state complete name to state abbrev. Dictionary
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "N.Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
    "United States Virgin Islands":  "VI",
    'British Columbia': "BC"
}

In [4]:
#US, state complete name to state abbrev. List
us_state = [
    "Alabama", "AL",
    "Alaska", "AK",
    "Arizona","AZ",
    "Arkansas", "AR",
    "California", "CA",
    "Colorado", "CO",
    "Connecticut", "CT",
    "Delaware", "DE",
    "Florida", "FL",
    "Georgia", "GA",
    "Hawaii", "HI",
    "Idaho", "ID",
    "Illinois", "IL",
    "Indiana", "IN",
    "Iowa", "IA",
    "Kansas", "KS",
    "Kentucky", "KY",
    "Louisiana", "LA",
    "Maine", "ME",
    "Maryland", "MD",
    "Massachusetts", "MA",
    "Michigan", "MI",
    "Minnesota", "MN",
    "Mississippi", "MS",
    "Missouri", "MO",
    "Montana", "MT",
    "Nebraska", "NE",
    "Nevada", "NV",
    'N.Nevada',
    "New Hampshire", "NH",
    "New Jersey", "NJ",
    "New Mexico", "NM",
    "New York", "NY",
    "North Carolina", "NC",
    "North Dakota", "ND",
    "Ohio", "OH",
    "Oklahoma", "OK",
    "Oregon", "OR",
    "Pennsylvania", "PA",
    "Rhode Island", "RI",
    "South Carolina", "SC",
    "South Dakota", "SD",
    "Tennessee", "TN",
    "Texas", "TX",
    "Utah", "UT",
    "Vermont", "VT",
    "Virginia", "VA",
    "Washington", "WA",
    "West Virginia", "WV",
    "Wisconsin", "WI",
    "Wyoming", "WY",
    "District of Columbia", "DC",
    "American Samoa", "AS",
    "Guam", "GU",
    "Northern Mariana Islands", "MP",
    "Puerto Rico", "PR",
    "United States Minor Outlying Islands", "UM",
    "U.S. Virgin Islands", "VI",
    'United States Virgin Islands',
    'British Columbia', "BC"
]

In [5]:
def uson(s):
    if ('US' in s) or ('USA' in s) or ('United States' in s):
        return s
    else:
        for x in s:
            if x in us_state:
                return s
                break
        return 'not_US'

In [6]:
df_t=df.iloc[180000:360000,:]
df_t['loc_l']=df_t["User's location"].apply(lambda x: x.split(','))
df_t['judge']=df_t['loc_l'].apply(lambda x: uson(x))
df_t=df_t.loc[df_t['judge']!='not_US']
df_t

,Date,Content,User's location,userid,loc_l,judge
262929,2022-07-11 18:12:21+00:00,i drink coffee too late and now i can’t sleep,"Nevada, USA",9.161833e+17,"[Nevada, USA]","[Nevada, USA]"
262960,2022-07-11 18:04:50+00:00,@Ondoism I can't sleep at 67 unfortunately. I...,"Montana, US",7.865693e+08,"[Montana, US]","[Montana, US]"
263025,2022-07-11 17:52:50+00:00,"Poor Kevin I so understand this tho, I have to...","New York, USA",2.163728e+09,"[New York, USA]","[New York, USA]"
263034,2022-07-11 17:49:15+00:00,@snowanaya1 @RAGGEDYROYAL Yes!! The worst. Lik...,United States,2.669940e+08,[United States],[United States]
263093,2022-07-11 17:34:46+00:00,idk what time zone my body is in anymore 😭 i'v...,"Florida, USA",1.165915e+08,"[Florida, USA]","[Florida, USA]"
...,...,...,...,...,...,...
521666,2022-08-04 15:06:49+00:00,That peach Red Bull is good asf but don’t keep...,"California, USA",1.321507e+09,"[California, USA]","[California, USA]"
521676,2022-08-04 15:00:33+00:00,I can't imagine telling someone else what they...,"Maine, USA",1.418103e+18,"[Maine, USA]","[Maine, USA]"
521691,2022-08-04 14:49:41+00:00,@DevGruGamer All good brother! If i had Insomn...,Utah,1.347804e+18,[Utah],[Utah]
521693,2022-08-04 14:47:35+00:00,I hate it when I do insomnia Wordle and then d...,Maryland,4.557160e+08,[Maryland],[Maryland]


In [7]:
def extract_us_state(x):
    if 'United Kingdom' in x:
        return 'not US'
    else:
        l=x.split(',')
        if len(l)<=1:
            if ('US' in l) or ('USA' in l) or ('United States' in l):
                return "USA"
            elif l[0] in us_state:
                if l[0] in us_state_to_abbrev.keys():
                    return us_state_to_abbrev[l[0]]
                else:
                    return l[0]
            else:
                return 'not_us'
        else:
            s1=l[-1].strip()
            if ('US' in s1) or ('USA' in s1) or ('United States' in s1):
                s2=l[-2].strip()
                if s2 in us_state:
                    if s2 in us_state_to_abbrev.keys():
                        return us_state_to_abbrev[s2]
                    else:
                        return s2
                elif bool(re.search('[0-9]+',s2))==True:
                    if len(l)>2:
                        for a in l[:-2]:
                            a=a.strip()
                            if a in us_state:
                                if a in us_state_to_abbrev.keys():
                                    return us_state_to_abbrev[a]
                                    break
                                else:
                                    continue
                        return "zip"+s2+" nfs"
                elif s2=='New England':
                    return ""+s2+' nfs'
                else:
                    return 'US'
            else:
                return 'not_us'

In [8]:
df_1_18=pd.read_csv('data/map_location_p1.csv')
df_1_18=df_1_18[['0']]
df_1_18=df_1_18.rename(columns={"0":'loc_all'})
df_1_18['loca_us']=df_1_18['loc_all'].apply(lambda x: uson(x))
df_1_18_us=df_1_18.loc[df_1_18['loca_us']!='not_US']
df_1_18_us=df_1_18_us[['loca_us']]
df_1_18_us['state']=df_1_18_us['loca_us'].apply(lambda x: extract_us_state(x))
df_1_18_us

,loca_us,state
1,"Philadelphia, Philadelphia County, Pennsylvani...",PA
2,"Myrtle Beach, Horry County, South Carolina, 29...",SC
3,"California, United States",CA
6,"Myrtle Beach, Horry County, South Carolina, 29...",SC
7,"Tampa, Hillsborough County, Florida, United St...",FL
...,...,...
64709,"17, Alviso, San Jose, Santa Clara County, Cali...",CA
64724,"Money Ruins Everything, 713, South Los Angeles...",CA
64727,"Catland, 987, Flushing Avenue, Brooklyn, Kings...",NY
64738,"California, United States",CA


In [9]:
def find_q(x):
    if "nfs" in x:
        return x
    else:
        return 'no'

In [10]:
df_1_18_us['State_Q']=df_1_18_us["state"].apply(lambda x: find_q(x))
df_1_18_us[df_1_18_us['State_Q']!='no']

,loca_us,state,State_Q
1719,"New England, United States",New England nfs,New England nfs
2134,"New England, United States",New England nfs,New England nfs
2348,"New England, United States",New England nfs,New England nfs
2694,"Wabash River, 46750, United States",zip46750 nfs,zip46750 nfs
2701,"New England, United States",New England nfs,New England nfs
2812,"New England, United States",New England nfs,New England nfs
2981,"New England, United States",New England nfs,New England nfs
3279,"New England, United States",New England nfs,New England nfs
3486,"New England, United States",New England nfs,New England nfs
3652,"New England, United States",New England nfs,New England nfs


In [11]:
df_1_18_us=df_1_18_us[df_1_18_us['State_Q']=='no']
df_1_18_us

,loca_us,state,State_Q
1,"Philadelphia, Philadelphia County, Pennsylvani...",PA,no
2,"Myrtle Beach, Horry County, South Carolina, 29...",SC,no
3,"California, United States",CA,no
6,"Myrtle Beach, Horry County, South Carolina, 29...",SC,no
7,"Tampa, Hillsborough County, Florida, United St...",FL,no
...,...,...,...
64709,"17, Alviso, San Jose, Santa Clara County, Cali...",CA,no
64724,"Money Ruins Everything, 713, South Los Angeles...",CA,no
64727,"Catland, 987, Flushing Avenue, Brooklyn, Kings...",NY,no
64738,"California, United States",CA,no


In [12]:
df_18_36=df_t
df_18_36_us=df_18_36[["User's location"]]
df_18_36_us=df_18_36_us.rename(columns={"User's location":'loca_us'})
df_18_36_us['state']=df_18_36_us['loca_us'].apply(lambda x: extract_us_state(x))
df_18_36_us

,loca_us,state
262929,"Nevada, USA",NV
262960,"Montana, US",MT
263025,"New York, USA",NY
263034,United States,USA
263093,"Florida, USA",FL
...,...,...
521666,"California, USA",CA
521676,"Maine, USA",ME
521691,Utah,UT
521693,Maryland,MD


In [13]:
df_18_36_us[df_18_36_us['state'].isna()]

,loca_us,state


In [14]:
df_18_36_us['State_Q']=df_18_36_us['state'].apply(lambda x: find_q(x))
df_18_36_us[df_18_36_us['State_Q']!='no']

,loca_us,state,State_Q


In [15]:
df_36_54=pd.read_csv('data/map_location_p3.csv')
df_36_54['loca_us']=df_36_54['location'].apply(lambda x: uson(x))
df_36_54_us=df_36_54.loc[df_36_54['loca_us']!='not_US']
df_36_54_us=df_36_54_us[['loca_us']]
df_36_54_us['state']=df_36_54_us['loca_us'].apply(lambda x: extract_us_state(x))
df_36_54_us

,loca_us,state
2,"Sunbeam, Jacksonville, Duval County, Florida, ...",FL
5,"Wellesley, Norfolk County, Massachusetts, Unit...",MA
14,"Grand Rapids, Kent County, Michigan, United St...",MI
16,"Virginia, United States",VA
17,"Devils Den, Town of Greenfield, Saratoga Count...",NY
...,...,...
121391,"Texas, United States",TX
121392,"Madison, Dane County, Wisconsin, United States",WI
121394,"Chicago, Cook County, Illinois, United States",IL
121395,"Atlanta, Fulton County, Georgia, 30303, United...",GA


In [16]:
df_36_54_us['State_Q']=df_36_54_us['state'].apply(lambda x: find_q(x))
df_36_54_us[df_36_54_us['State_Q']!='no']

,loca_us,state,State_Q
1386,"New England, United States",New England nfs,New England nfs
3302,"New England, United States",New England nfs,New England nfs
4956,"New England, United States",New England nfs,New England nfs
5129,"New England, United States",New England nfs,New England nfs
8517,"New England, United States",New England nfs,New England nfs
...,...,...,...
117900,"New England, United States",New England nfs,New England nfs
119865,"New England, United States",New England nfs,New England nfs
120354,"New England, United States",New England nfs,New England nfs
120623,"New England, United States",New England nfs,New England nfs


In [17]:
df_36_54_us=df_36_54_us[df_36_54_us['State_Q']=='no']
df_36_54_us

,loca_us,state,State_Q
2,"Sunbeam, Jacksonville, Duval County, Florida, ...",FL,no
5,"Wellesley, Norfolk County, Massachusetts, Unit...",MA,no
14,"Grand Rapids, Kent County, Michigan, United St...",MI,no
16,"Virginia, United States",VA,no
17,"Devils Den, Town of Greenfield, Saratoga Count...",NY,no
...,...,...,...
121391,"Texas, United States",TX,no
121392,"Madison, Dane County, Wisconsin, United States",WI,no
121394,"Chicago, Cook County, Illinois, United States",IL,no
121395,"Atlanta, Fulton County, Georgia, 30303, United...",GA,no


In [18]:
df_1_54=pd.concat([df_1_18_us,df_18_36_us,df_36_54_us])
df_1_54=df_1_54[df_1_54['State_Q']=='no']
df_1_54=df_1_54[['state']]

In [19]:
df_1_54.to_csv('crawled_us_state.csv')

#### Draw the map with plotly

In [20]:
state_pop=pd.read_csv('data/state_population.csv')
state_pop=state_pop[['NAME','ESTIMATESBASE2020']]
state_pop
# state_pop=state_pop.to_dict('split')
# state_pop['data']
s_pop_d={}
state_pop.iloc[0,1]
for i in range(len(state_pop)):
    try:
        state_pop.iloc[i,0]=us_state_to_abbrev[state_pop.iloc[i,0]]
    except:
        continue
state_pop=pd.DataFrame(state_pop.iloc[5:,:])
#divided by 1000, make the number not too large
state_pop['POP']=round(state_pop['ESTIMATESBASE2020']/1000,2)
state_pop
for i in range(len(state_pop)):
    s_pop_d[state_pop.iloc[i,0]]=state_pop.iloc[i,2]
s_pop_d

{'AL': 5024.28,
 'AK': 733.39,
 'AZ': 7151.5,
 'AR': 3011.52,
 'CA': 39538.22,
 'CO': 5773.71,
 'CT': 3605.94,
 'DE': 989.95,
 'DC': 689.54,
 'FL': 21538.19,
 'GA': 10711.91,
 'HI': 1455.27,
 'ID': 1839.11,
 'IL': 12812.51,
 'IN': 6785.53,
 'IA': 3190.37,
 'KS': 2937.88,
 'KY': 4505.84,
 'LA': 4657.76,
 'ME': 1362.36,
 'MD': 6177.22,
 'MA': 7029.92,
 'MI': 10077.33,
 'MN': 5706.49,
 'MS': 2961.28,
 'MO': 6154.91,
 'MT': 1084.22,
 'NE': 1961.5,
 'NV': 3104.61,
 'NH': 1377.53,
 'NJ': 9288.99,
 'NM': 2117.52,
 'NY': 20201.25,
 'NC': 10439.39,
 'ND': 779.09,
 'OH': 11799.45,
 'OK': 3959.35,
 'OR': 4237.26,
 'PA': 13002.7,
 'RI': 1097.38,
 'SC': 5118.42,
 'SD': 886.67,
 'TN': 6910.84,
 'TX': 29145.5,
 'UT': 3271.62,
 'VT': 643.08,
 'VA': 8631.39,
 'WA': 7705.28,
 'WV': 1793.72,
 'WI': 5893.72,
 'WY': 576.85,
 'PR': 3285.87}

In [21]:
def convert_c_p(x):
    try:
        a=float(s_pop_d[x])
    except:
        a='??'
    return a

In [22]:
map = pd.read_csv('data/crawled_us_state.csv')
map = map[['state']]
map['count']=1
map=map.groupby(['state']).sum()
map=map.reset_index()
map['state_pop']=map['state'].apply(lambda x: convert_c_p(x))
map=map[map['state_pop']!='??']
map['count/pop(K)']=0
for i in range(len(map)):
    map.iloc[i,3]=round(float(map.iloc[i,1])/float(map.iloc[i,2]),2)
map

,state,count,state_pop,count/pop(K)
0,AK,237,733.39,0.32
1,AL,1168,5024.28,0.23
2,AR,499,3011.52,0.17
4,AZ,1895,7151.5,0.26
6,CA,13117,39538.22,0.33
7,CO,1559,5773.71,0.27
8,CT,1013,3605.94,0.28
9,DC,1253,689.54,1.82
10,DE,211,989.95,0.21
11,FL,6191,21538.19,0.29


#### Map code

We take log of the count/pop(K) to make the scale more smooth<br>
Also, we set the mid point of the continuous scale to be more close to the most saturated blue to make the area more distinguishable. 

In [27]:
fig = px.choropleth(map, locations = 'state', scope = 'usa', locationmode="USA-states", color=np.log10(map['count/pop(K)']),
                           color_continuous_scale=[[0.0, "rgb(255,255,255)"],
                [0.5, "rgb(108,161,255)"],
                [1.0, "rgb(3,94,255)"]], 
                # hover_name='state', hover_data=['state_pop','count/pop(K)']
                custom_data=map[['state','count/pop(K)']],
                labels={'color': 'count/pop(K)'}
)
fig.add_scattergeo(
    locations=map['state'],
    locationmode='USA-states',
    text=map['state'],hoverinfo = 'none',
    mode='text')
fig.update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})
fig.update_layout(geo=dict(bgcolor= 'rgba(0,0,0,0)'))
fig.update_traces(customdata=map[['state','count/pop(K)','count']],
                  hovertemplate="%{customdata[0]} <br>Count: %{customdata[2]} <br>Count/Pop(K): %{customdata[1]} ")
fig.update_layout(
    title={
        'text': "Geological Distribution of Sleepless Twitter Users in the US",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [28]:
fig.write_html("USA_map.html") 